In [86]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn import svm

import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from config.get import cfg

In [58]:
X_train = np.load(cfg['files']['encoded_train_features'])
X_test  = np.load(cfg['files']['encoded_test_features'])

train_ids = np.load(cfg['files']['train_ids']).astype(int)
test_ids = np.load(cfg['files']['test_ids']).astype(int)
train_ids = pd.DataFrame({"cycle_id":train_ids})
test_ids  = pd.DataFrame({"cycle_id":test_ids})


target = pd.read_csv(cfg['files']['target'])

y_train = train_ids.join(target,on="cycle_id",lsuffix="_").profitability
y_test = test_ids.join(target,on="cycle_id",lsuffix="_").profitability

In [59]:
scaler = StandardScaler()
scaler.fit(X_train)
tX_train = scaler.transform(X_train)
tX_test  = scaler.transform(X_test)


In [61]:
print(y_train.mean()) # imbalanced classes

0.9444980177863496


In [79]:
model = LogisticRegressionCV(cv=5,Cs=np.logspace(-4,4,10),class_weight="balanced",max_iter=1000)

In [80]:
model.fit(tX_train, y_train)

LogisticRegressionCV(Cs=array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]),
                     class_weight='balanced', cv=5, max_iter=1000)

In [82]:
model.scores_

{True: array([[0.46331012, 0.49919657, 0.50348152, 0.56400643, 0.57150509,
         0.568827  , 0.56668452, 0.56668452, 0.56668452, 0.56668452],
        [0.46170327, 0.48473487, 0.54793787, 0.5661489 , 0.57846813,
         0.58275308, 0.5832887 , 0.5832887 , 0.5832887 , 0.5832887 ],
        [0.44991966, 0.51794322, 0.53240493, 0.55704339, 0.568827  ,
         0.57257633, 0.56989823, 0.56989823, 0.56989823, 0.56989823],
        [0.46998928, 0.4721329 , 0.49249732, 0.55466238, 0.56270096,
         0.56538049, 0.5664523 , 0.5659164 , 0.5664523 , 0.5664523 ],
        [0.42282958, 0.48928189, 0.54930332, 0.56859593, 0.57395498,
         0.58038585, 0.5846731 , 0.5846731 , 0.5846731 , 0.5846731 ]])}

In [83]:
pred = model.predict(tX_test)

In [84]:
tn, fp, fn, tp = confusion_matrix(y_test,pred).ravel()
print(tn, fp, fn, tp)

112 104 1544 2241


In [85]:
f1_score(y_test,pred)

0.7311582381729201

# SVM

In [88]:

svm_parameters = {'kernel':('linear', 'rbf','poly'), 'C':np.logspace(-4,4,10)}
svc = svm.SVC()
base_lr = GridSearchCV(svc, svm_parameters,verbose=1,cv = 5)

In [89]:
base_lr.fit(tX_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
svm_pred = model.predict(tX_test)